In [239]:
import torch
import torch.nn as nn
import numpy as np

In [240]:
# We want a network that takes in a 5x1 vector and outputs single scalar value
# It should consist of only convolutional layers

NUM_POINTS = 10000
POINT_DIM = 5

SEARCH_RADIUS = 10000

In [241]:
class Net(nn.Module):
    def __init__(self, hidden_size=10):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 3, 2, 1, 0)
        self.conv2 = nn.Conv1d(3, 3, 2, 1, 0)
        self.conv3 = nn.Conv1d(3, 3, 2, 1, 0)
        self.conv4 = nn.Conv1d(3, 1, 2, 1, 0)
        self.initialise_weights()

    def initialise_weights(self):
        # We want the weights to be quite small
        self.conv1.weight.data.normal_(0, 0.1)
        self.conv2.weight.data.normal_(0, 0.1)
        self.conv3.weight.data.normal_(0, 0.1)
        self.conv4.weight.data.normal_(0, 0.1)

    def forward(self, x):
        x = x.view(-1, 1, POINT_DIM)
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.conv4(x)
        x = nn.functional.relu(x)
        return x.view(-1)


model = Net()
print(model)

example_input = torch.randn(4, POINT_DIM)
print(example_input)

output = model(example_input)
print(output)

Net(
  (conv1): Conv1d(1, 3, kernel_size=(2,), stride=(1,))
  (conv2): Conv1d(3, 3, kernel_size=(2,), stride=(1,))
  (conv3): Conv1d(3, 3, kernel_size=(2,), stride=(1,))
  (conv4): Conv1d(3, 1, kernel_size=(2,), stride=(1,))
)
tensor([[-0.7247,  0.0566,  0.1982, -0.1209,  0.3277],
        [-0.6003, -1.0274, -0.4787, -1.4924,  0.4707],
        [-0.8435, -1.3829, -1.1964, -0.2006,  0.8435],
        [-0.3924, -0.7674,  1.5255,  2.3943, -0.1622]])
tensor([0., 0., 0., 0.], grad_fn=<ViewBackward0>)


In [242]:
def sample_spherical(npoints, ndim):
    vec = np.random.randn(ndim, npoints)
    vec /= np.linalg.norm(vec, axis=0)
    return torch.Tensor(vec.T)


def sample_from_ball(npoints, ndim, radius):
    vec = np.random.randn(ndim, npoints)
    vec /= np.linalg.norm(vec, axis=0)
    radii = np.random.uniform(0, radius, npoints)
    vec *= radii
    return torch.Tensor(vec.T)


points = sample_from_ball(npoints=NUM_POINTS, ndim=POINT_DIM, radius=SEARCH_RADIUS)
print(torch.linalg.norm(points[59]))

# We want to sort the coordinates of each point so that they are always increasing
sorted_points = torch.sort(points, dim=1).values
print(sorted_points[5])

tensor(7812.1396)
tensor([-838.1958, -115.7162,  -22.4317,   -5.0522,   86.8268])


In [243]:
# example = points[:3]
# example.requires_grad = True
# print(example)

# output = model(example)
# print(output)
# output.backward(torch.ones_like(output))

# jacobians = example.grad
# print(jacobians)

In [244]:
def get_jacobians(points, model):
    """
    Compute the Jacobian of the model outputs with respect to its inputs for each input point.
    
    :param points: A tensor of shape (N, M) containing N points of dimension M.
    :param model: A PyTorch model that accepts inputs of shape (N, M) and outputs a tensor of shape (N, 1).
    :returns: A tensor of Jacobians of shape (N, M), where each row corresponds to the Jacobian of the model output with respect to the input point.
    """
    points.requires_grad = True
    output = model(points)
    output.backward(torch.ones_like(output))
    jacobians = points.grad
    return jacobians


def round_tensor(tensor, decimal_places):
    return torch.round(tensor * 10**decimal_places) / 10**decimal_places


def remove_duplicate_tensors(tensor):
    return torch.unique(tensor, dim=0)


jacobians = get_jacobians(points, model)
print(jacobians.shape)

unique_jacobians = remove_duplicate_tensors(round_tensor(jacobians, 10))
print(unique_jacobians.shape)

torch.Size([10000, 5])
torch.Size([1349, 5])


In [245]:
print(jacobians[4])

tensor([ 3.7209e-05,  5.1341e-05, -7.7134e-04, -3.7868e-04,  5.3303e-05])
